# Pruebas por pares o _pairwise testing_

[Juan M. Fonseca-Solís](https://juanfonsecasolis.github.io) · Setiembre 2020 · 5 min read

---

## Resumen
A diferencia del resto de artículos de este blog, el presente ipython notebook no contiene ningún componente original, sino que pretendemos simplemente rescatar la descripción matemática y utilidad de una técnica de estimación de casos de prueba llamada _pruebas por pares_. Los resultados obtenidos muestran que se puede lograr una buena cobertura en tiempo $O(op)$.

## Introducción

Como su nombre lo indica, la técnica de por pares construye escenarios de prueba tomando pares de parámetros en lugar de hacer una combinación exhaustiva de variables. Esto lo realiza asumiendo como verdadera la premisa de Kuhn et. al [1,4]:

_"**Regla de interacción:** la mayoría de las fallas son producidas por un solo factor o por la interación entre dos factores, y se observan progresivamente menos fallas producidas por la interacción entre tres o más factores."_

## Pseudo-formalismo matemático

Matemáticamente la técnica de la prueba por pares se puede expresar como sigue. Sea $\{P_1, P_2, \ldots, P_N\} \subset \mathbb{C}^{\max\{\lvert{R_i}\rvert\}}$ un espacio vectorial generado por un sistema de N parámetros, el conjunto de valores que puede tomar cada parámetro $R(P_i) = R_i$ es denominado el rango de un parámetro [1]. Es decir, que si hicieramos el modelado de un interruptor de la casa, los posibles valores de prueba serían:

$$
R_i = \{\text{encendido}, \text{apagado}\},
$$

donde la cardinalidad del conjunto, $\lvert{R_i}\rvert=n_i$, sería igual a 2. Si modelásemos todos los interruptores de la casa y nos interesara el estado de la casa en ese momento, la combinación de los valores posibles sería un producto, es decir, $\prod{n_i}$. Podría suceder, sin embargo, que no todos los encendedores tuvieran dos estados, sino tres, por ejemplo: encendido alto, encendido bajo, apagado; en cuyo caso $2^N < \prod{n_i}$.

Si en lugar de hacer una combinación exhaustiva usaramos la técnica de prueba por pares tendríamos que definir $X=\{n_i\}$ como el multiconjunto que contiene todos los posibles valores de todos los parámetros, por ejemplo, $X=\{2, 2, 3, \ldots\}$. Se dice que este es un multiconjunto o _multiset_, porque podemos tener elementos repetidos [2]. Luego tendríamos que el número de casos de prueba $T$ sería [1]:

$$
T = \max\{X\} \max\{X/\max\{X\}\},
$$

en otras palabras, el producto de los rangos de los dos parámetros con la mayor cantidad de valores posibles. 

## Recetario

Con base en este formalismo matemático es posible resumir la técnica de pruebas de pares como:
1. Listar todos los $N$ parámetros del sistema.
2. Identificar para cada parámetro, el número posible de valores.
3. Tomar los dos parámetros $i,j $ cuyo $\lvert{R(\cdot)}\rvert$ sea mayor al resto.
4. Construir una tabla de $n_i n_j$ filas y $N$ columnas.
5. Completar las columnas haciendo la combinación de los valores de los parámetros $i,j$ y rellenar el resto de columnas como haciendo una combinatoria completa.

## Ejemplo práctico
Veamos un ejemplo con $P_1 = \{T,F\}$, $P_2 = \{1,2,3\}$, $P_3 = \{a,b,c,d\}$ y $P_4 = \{x,y\}$:

|    | Valor 1 | Valor 2 | Valor 3 | Valor 4 | $\lvert{R_i}\rvert$ |
|----|--------|--------|--------|--------|---------------------|
| $P_1$ | T      | F      | -      | -      | 2                   |
| $P_2$ | 1      | 2      | 3      | -      | 3                   |
| $P_3$ | a      | b      | c      | d      | 4                   |
| $P_4$ | x      | y      | -      | -      | 2                   |

Los parámetros con más valores serían $P_3$ y $P_2$, y el producto de las cardinalidades de los conjuntos de sus valores sería $\max\{X\} \max\{X/\max\{X\}\} = 4 \cdot 3 = 12$. Con base en esto, podemos armar una tabla de combinaciones truncada como sigue:

|CP/P| $P_1$ | $P_2$ | $P_3$ | $P_4$ |
|----|-------|-------|-------|-------|
| 1  | a     | 1     | T     | x     |
| 2  | b     | 2     | F     | y     |
| 3  | c     | 3     | T     | x     |
| 4  | d     | 1     | F     | y     |
| 5  | a     | 2     | T     | x     |
| 6  | b     | 3     | F     | z     |
| 7  | c     | 1     | T     | x     |
| 8  | d     | 2     | F     | z     |
| 9  | a     | 3     | T     | x     |
| 10 | b     | 1     | F     | z     |
| 11 | c     | 2     | T     | x     |
| 12 | d     | 3     | F     | z     |

Si hubiesemos hecho la combinación completa de todos los valores, esto nos habría tomado $\prod\{n_i\}=4 \cdot 3 \cdot 2 \cdot 3 = 48$ casos de prueba, lo cual representa un esfuerzo considerable. Por eso se dice la prueba por pares tomar tiempo $O(op)$, con $o,p \in \{n_i\}$, en lugar de $\prod{n_i}$.

# Automatización

Aunque más rápida que una combinatoria completa, las pruebas por pares requieren sentarse con papel y lápiz a escribir la tabla. Podemos obtener el mismo resultado implementando al algoritmo en la computadora, o incluso más rápido todavía, usando una herramienta de Microsoft llamada PICT (disponible gratuitamente en el sitio [https://pairwise.yuuniworks.com](https://pairwise.yuuniworks.com)) [3]. El comando para este programa es `pict modelo.txt [opciones]`.

modelo.txt:
```
P1: T,F
P2: 1,2,3
P3: a,b,c,d
P4: x,y
```

Salida:
```
P1	P2	P3	P4
T	2	c	y
T	3	a	x
F	1	c	x
F	2	a	y
T	1	b	y
F	3	c	y
F	3	b	x
F	2	d	x
T	1	a	y
T	3	d	y
T	1	d	y
F	2	b	x
```

Lo cual corresponde en efecto, a los 12 CP que calculamos a mano.

# Conclusiones

La técnica de las pruebas por pares permite estimar casos de prueba identificando los dos parámetros del sistema con el rango de valores más amplio. Las combinaciones generadas son mucho menores que las del enfoque exhaustivo, pero esto no representa una desventaja en términos de cobertura, pues según el principio de Kuhn et al. la mayoría de pulgas involucran dos parámetros o menos. Existen herramientas como el PICT de Microsoft que permiten automatizar el proceso de generación de casos de prueba, haciendo que esta técnica sea fácil de usar en el corto tiempo que se dispone en el trabajo.

# Referencias:
1. Wikipedia contributors. (2020, July 8). All-pairs testing. In Wikipedia, The Free Encyclopedia. Retrieved 20:09, September 6, 2020, from https://en.wikipedia.org/w/index.php?title=All-pairs_testing&oldid=966710808.
2. Multiconjunto. (2019, 11 de octubre). Wikipedia, La enciclopedia libre. Fecha de consulta: 20:31, septiembre 6, 2020 desde https://es.wikipedia.org/w/index.php?title=Multiconjunto&oldid=120189103. 
3. J McCaffrey. Test Run - Pairwise Testing with QICT. Volume 24 Number 12. Dec 2009. URL: https://docs.microsoft.com/en-us/archive/msdn-magazine/2009/december/test-run-pairwise-testing-with-qict
4. D. Richard Kuhn, Raghu N. Kacker, Yu Lei. Practical combinatorial Testing. NIST Special Publication 800-142. Departamento de Comercio de Estados Unidos. URL: https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-142.pdf (consultado por última vez el 09/09/20).

---
<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Licencia de Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br/>Este obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">licencia de Creative Commons Reconocimiento-NoComercial-SinObraDerivada 4.0 Internacional</a>. El sitio [juanfonsecasolis.github.io](https://juanfonsecasolis.github.io) es un blog dedicado a la investigación independiente en temas relacionados al procesamiento digital de señales. Para reutilizar este artículo y citar las fuente por favor utilice el siguiente Bibtex:

```
@online{Fonseca2020,
  author = {Juan M. Fonseca-Solís},
  title = { Pruebas por pares o pairwise testing},
  year = 2020,
  url = {https://juanfonsecasolis.github.io/blog/JFonseca.pairwisetesting.html},
  urldate = {}
}
```